# Homework 6

This homework is based on the Boston data.
<br>Fit "the biggest model" (multiple linear regression) that you possibly can (including interactions and categorical variables).
<br>Dependent Variable: `MEDV`
<br>
<br>Whoever achieves the highest $R^2$ wins :)


-----------------
**Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns
import scipy

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Import the dataset

Dataset Description:
The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic prices and the demand for clean air'.

Features Description:
- **CRIM**: per capita crime rate by town
- **ZN**: proportion of residential land zoned for lots over 25,000 sq.ft.
- **INDUS**: proportion of non-retail business acres per town
- **CHAS**: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- **NOX**: nitric oxides concentration (parts per 10 million)
- **RM**: average number of rooms per dwelling
- **AGE**: proportion of owner-occupied units built prior to 1940
- **DIS**: weighted distances to five Boston employment centres
- **RAD**: index of accessibility to radial highways
- **TAX**: full-value property-tax rate per 10,000 dollars
- **PTRATIO**: pupil-teacher ratio by town
- **BLACK**: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- **LSTAT**: percentage lower status of the population
- **MEDV**: Median value of owner-occupied homes in 1000's dollars

In [2]:
boston = pd.read_csv(r"C:\Users\svawe\OneDrive\Desktop\Master\DataScience_01\data\Boston.csv")

# independent variables
X = boston.drop('medv', axis=1)

# dependent variable
y = boston['medv']

boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
boston.shape

(506, 14)

In [4]:
boston.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## Solution 1:
we are talking about R²? Its biased towards the number of predictors -> just throw all of it in

In [5]:
#actual model
est = smf.ols('medv ~ crim + zn + indus + chas + nox + rm + age + dis + rad + tax + ptratio + black + lstat', boston).fit()
est.summary().tables[0]

Dep. Variable:,medv,R-squared:,0.741
Model:,OLS,Adj. R-squared:,0.734
Method:,Least Squares,F-statistic:,108.1
Date:,"Tue, 16 Nov 2021",Prob (F-statistic):,6.72e-135
Time:,15:06:36,Log-Likelihood:,-1498.8
No. Observations:,506,AIC:,3026.
Df Residuals:,492,BIC:,3085.
Df Model:,13,,
Covariance Type:,nonrobust,,


R² of 0.741 and R² adj of 0.734 is pretty good!

In [6]:
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,36.4595,5.103,7.144,0.000,26.432,46.487
crim,-0.1080,0.033,-3.287,0.001,-0.173,-0.043
zn,0.0464,0.014,3.382,0.001,0.019,0.073
indus,0.0206,0.061,0.334,0.738,-0.100,0.141
chas,2.6867,0.862,3.118,0.002,0.994,4.380
nox,-17.7666,3.820,-4.651,0.000,-25.272,-10.262
rm,3.8099,0.418,9.116,0.000,2.989,4.631
age,0.0007,0.013,0.052,0.958,-0.025,0.027
dis,-1.4756,0.199,-7.398,0.000,-1.867,-1.084
rad,0.3060,0.066,4.613,0.000,0.176,0.436


targets for dummy coding: only rad

## Solution 2: Incorporate dummy coding to maybe increase R²

In [7]:
boston = boston.astype(dtype = {"rad":"category"})

In [8]:
 est = smf.ols('medv ~ crim + zn + chas + nox + rm + dis + rad + tax + ptratio + black + lstat', boston).fit()
est.summary().tables[0]

Dep. Variable:,medv,R-squared:,0.750
Model:,OLS,Adj. R-squared:,0.741
Method:,Least Squares,F-statistic:,81.08
Date:,"Tue, 16 Nov 2021",Prob (F-statistic):,1.10e-133
Time:,15:06:36,Log-Likelihood:,-1489.7
No. Observations:,506,AIC:,3017.
Df Residuals:,487,BIC:,3098.
Df Model:,18,,
Covariance Type:,nonrobust,,


The R² increased to 0.75, corrected R² at 0.741 -> I could just go ahead and make every variable caterogical, and it will likely increase the R².But I wont do it since all of the other variables are either dichotom or metric, which are fitted for a regression.

## Solution 3:
Lets try to optimize  the adjusted R² using the output of first regression and check what variables are not (as) significant!

It seems like particulary the "indus" and "age" variable are insignificant, lets exclude these two first. 
While it is to be expected that R² is reduced by it, the adjusted R² may rise.

In [9]:
 est = smf.ols('medv ~ crim + zn + chas + nox + rm + dis + rad + tax + ptratio + black + lstat', boston).fit()
est.summary().tables[0]

Dep. Variable:,medv,R-squared:,0.750
Model:,OLS,Adj. R-squared:,0.741
Method:,Least Squares,F-statistic:,81.08
Date:,"Tue, 16 Nov 2021",Prob (F-statistic):,1.10e-133
Time:,15:06:36,Log-Likelihood:,-1489.7
No. Observations:,506,AIC:,3017.
Df Residuals:,487,BIC:,3098.
Df Model:,18,,
Covariance Type:,nonrobust,,


The adjusted R² and the R² did not change

In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [24]:
X = boston[["crim", "zn", "chas", "nox", "rm", "dis" , "rad" , "tax" , "ptratio" , "black" , "lstat"]]
y = boston.medv
# Create training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

In [44]:
regr = LinearRegression()
regr.fit(X_train, y_train)
yHat_test = regr.predict(X_test)

MSE_test = np.mean((yHat_test - y_test)**2)
R2_test = 1 - MSE_test / np.var(y_test)


yHat_train = regr.predict(X_train)

MSE_train = np.mean((yHat_train - y_train)**2)
R2_train = 1 - MSE_train / np.var(y_train)

import sklearn.metrics as metrics
metrics.mean_squared_error(yHat_train, y_train), MSE_test, R2_test, MSE_train, R2_train

(21.87170961197803,
 23.362972267101487,
 0.7635982049770118,
 21.871709611977998,
 0.7292481475098824)

In [25]:
X_train.head()

,crim,zn,chas,nox,rm,dis,rad,tax,ptratio,black,lstat
42,0.14150,0.0,0,0.448,6.169,5.7209,3,233,17.9,383.37,5.81
58,0.15445,25.0,0,0.453,6.145,7.8148,8,284,19.7,390.68,6.86
385,16.81180,0.0,0,0.700,5.277,1.4261,24,666,20.2,396.90,30.81
78,0.05646,0.0,0,0.437,6.232,5.0141,5,398,18.7,386.40,12.34
424,8.79212,0.0,0,0.584,5.565,2.0635,24,666,20.2,3.65,17.16


In [26]:
from sklearn.model_selection import cross_val_score

reg_all = LinearRegression()

cv_results = cross_val_score(reg_all, X, y, cv=5)

print("CV results: ", cv_results)
print("CV results mean: ", np.mean(cv_results))

CV results:  [ 0.64868265  0.75788146  0.59808111  0.15142378 -0.23839972]
CV results mean:  0.38353385911170523
